# MNISTによる分類問題

## ライブラリのインポート

In [1]:
import numpy as np
import tensorflow as tf

# tensorflow-datasetsモジュールを使ってMNISTデータセットをインポートしていきます。まだインストールしていない場合は以下のコマンドでインストールしましょう。
# pip install tensorflow-datasets 
# conda install tensorflow-datasets

import tensorflow_datasets as tfds

# このデータセットは、C:\Users\*USERNAME*\tensorflow_datasets\...に保存されます。

2024-02-10 17:08:57.907866: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## データの読み込み


In [2]:
# tfds.loadを使ってデータセットを読み込んでいきます 
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# with_info=True をすることによって、データに関する情報を得ることができるようになります。

# as_supervised=Trueをすることによって、データを入力とターゲットの二つののタプル形式で作成することができます 

# 訓練用データとテスト用データをそれぞれの変数に代入します
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# ここから検証用データセットを作っていきます。まずは、検証用データに割り当てる割合を決めます。
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
# 数字を整数に変換していきます（既に整数になっている場合も念のため行います）
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# テストデータの数も変数に入れていきます
num_test_samples = mnist_info.splits['test'].num_examples
# 数字を整数に変換していきます
num_test_samples = tf.cast(num_test_samples, tf.int64)


# 元の入力データの範囲である0から255を0と1に変えていきます
# 今回は、変換するための関数を作成していきます
def scale(image, label):
    # 数字が小数であることを確認します
    image = tf.cast(image, tf.float32)
    # 0と1の範囲におさめるため、255で割っていきます 
    image /= 255.

    return image, label


# mapメソッドを使ってデータの変換を行っていきます
scaled_train_and_validation_data = mnist_train.map(scale)

# テストデータに関しても同様の変換を行っていきます
test_data = mnist_test.map(scale)

#データをシャッフルするためのバッファのサイズを決めていきます
BUFFER_SIZE = 10000

# シャッフルメソッドを使ってデータをシャッフルしていきます
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# takeメソッドを使ってシャッフルした検証用データを変数に代入していきます
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

# skipメソッドを使って訓練用データセットを変数に代入します
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# batchの大きさを定義します
BATCH_SIZE = 100

# 訓練データをバッチの数毎に分けていきます
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

# テストデータに関してもバッチ毎に分けていきます
test_data = test_data.batch(num_test_samples)


# バリデーションデータセットに関し、入力とターゲットそれぞれの変数を作成していきます
validation_inputs, validation_targets = next(iter(validation_data))

2024-02-10 17:09:11.720415: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
